In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, log_loss
)
import numpy as np

In [ ]:
data = pd.read_csv("../../Data/BRFSS_2024_model_ready.csv", low_memory=False)

In [15]:
X = data.drop('DIABETE4', axis=1)
y = data['DIABETE4'].astype(int)

KeyError: "['DIABETE4'] not found in axis"

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

NameError: name 'X' is not defined

In [24]:
nb = GaussianNB()
nb.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [12]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Hyperparameter search for GaussianNB
param_grid = {
    "var_smoothing": np.logspace(-12, -6, 13)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = GridSearchCV(
    GaussianNB(),
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

search.fit(X_train, y_train)

print("Best CV macro-F1:", search.best_score_)
print("Best parameters:", search.best_params_)

# Best model
best_nb = search.best_estimator_

# Evaluate
y_pred = best_nb.predict(X_test)
y_proba = best_nb.predict_proba(X_test)

print("\n=== Tuned GaussianNB Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro Precision:", precision_score(y_test, y_pred, average='macro', zero_division=0))
print("Macro Recall:", recall_score(y_test, y_pred, average='macro', zero_division=0))
print("Macro F1:", f1_score(y_test, y_pred, average='macro', zero_division=0))
print("Log Loss:", log_loss(y_test, y_proba))

print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


NameError: name 'X_train' is not defined

In [6]:
y_pred = nb.predict(X_test)
y_proba = nb.predict_proba(X_test)

In [7]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
logloss = log_loss(y_test, y_proba)

In [8]:
print(f"Accuracy:           {accuracy:.4f}")
print(f"Precision (macro):  {precision:.4f}")
print(f"Recall (macro):     {recall:.4f}")
print(f"F1 Score (macro):   {f1:.4f}")
print(f"Log Loss:          {logloss:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy:           0.6864
Precision (macro):  0.4199
Recall (macro):     0.4714
F1 Score (macro):   0.4234
Log Loss:          6.0492

Classification Report:
               precision    recall  f1-score   support

           1       0.31      0.52      0.39     13162
           3       0.90      0.73      0.81     75226
           4       0.05      0.16      0.08      2261

    accuracy                           0.69     90649
   macro avg       0.42      0.47      0.42     90649
weighted avg       0.79      0.69      0.73     90649


Confusion Matrix:
 [[ 6822  4844  1496]
 [14563 55026  5637]
 [  794  1095   372]]
